In [4]:
import tensorflow as tf
import numpy as np
import gym
import gym_pull]\

ModuleNotFoundError: No module named 'gym_pull'

In [3]:
env = gym.make('SuperMarioBros-1-1-v0')
env = env.unwrapped

UnregisteredEnv: No registered env with id: SuperMarioBros-1-1-v0

# 超參數

In [ ]:
BATCH_SIZE = 32
EPISILON = 0.9
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
MEMORY_CAPACITY = 150
GAMMA = 0.9
LR = 0.01

In [54]:
tf.reset_default_graph()

In [55]:
class dueling_double_DQN_Agent():
    def __init__(self,
                 n_actions = 2,
                 n_states = 4,
                 memory_capacity = 150,
                 episilon = 0.9,
                 gamma = 0.9,
                 batch_size = 32,
                 lr = 0.01,
                 memory_counter = 0,
                 synchronize_iter = 100,
                 
                 ):
        
        self.memory = np.zeros((memory_capacity,n_states*2+2))
        self.memory_capacity = memory_capacity
        self.memory_counter = 0
        self.synchronize_iter = synchronize_iter
        self.synchronize_counter = 0
        self.batch_size = batch_size
        self.n_states = n_states
        self.n_actions = n_actions
        self.gamma = gamma
        self.lr = lr
        self.episilon = episilon
        
        self._build_net()
        
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
        
    def _build_net(self):
        self.s = tf.placeholder(tf.float32,[None,self.n_states],name = "current_state")
        self.a = tf.placeholder(tf.int32,[None,],name = "action")
        self.r = tf.placeholder(tf.float32,[None,] ,name = "reward")
        self.s_ = tf.placeholder(tf.float32,[None,self.n_states],name = "next_state")
        
        init_weight,init_bias = tf.truncated_normal_initializer(stddev=0.01),tf.constant_initializer(0.1)
        
        
        ######## dueling 要改網路架構  ########
        
        # eval_net
        with tf.variable_scope("eval_net"):
            
            self.e_fc1 = tf.layers.dense(self.s , 50 , activation = tf.nn.relu , 
                            kernel_initializer = init_weight , 
                            bias_initializer = init_bias, name="e_fc1")
            
            self.e_value = tf.layers.dense(self.e_fc1 , 1 , activation = tf.nn.relu , 
                            kernel_initializer = init_weight , 
                            bias_initializer = init_bias, name="e_value")
            
            self.e_act_q = tf.layers.dense(self.e_fc1 , self.n_actions , activation=tf.nn.relu,
                                   kernel_initializer = init_weight,
                                   bias_initializer = init_bias , 
                                    name="e_fc2")
            
            
            
            
        # target_net
        with tf.variable_scope("target_net"):
            self.t_fc1 = tf.layers.dense(self.s_ , 50 , activation =tf.nn.relu,
                                        kernel_initializer = init_weight , 
                                        bias_initializer = init_bias , name="t_fc1")
            
            self.t_value = tf.layers.dense(self.t_fc1 , 1 , activation = tf.nn.relu , 
                kernel_initializer = init_weight , 
                bias_initializer = init_bias, name="t_value")
            
            self.t_act_q = tf.layers.dense(self.t_fc1 , self.n_actions , activation=tf.nn.relu,
                                        kernel_initializer = init_weight,
                                        bias_initializer = init_bias , name= "t_fc2")
        
        
        ######################################
        
        
        
        
        self.eval_params = tf.get_collection(key = tf.GraphKeys.TRAINABLE_VARIABLES , scope = "eval_net")
        self.target_params = tf.get_collection(key = tf.GraphKeys.TRAINABLE_VARIABLES , scope = "target_net")
        
        
        # eval 參數同步到 target的op
        self.synchronize_op = [ tf.assign(t,e) for e,t in zip(self.eval_params,self.target_params)]
        
        # loss
        e_m = tf.reduce_mean(self.e_act_q, axis=1, keep_dims=True)
        e_devs_squared = tf.square(self.e_act_q - e_m)
        e_std = tf.reduce_mean(e_devs_squared, axis=1, keep_dims=True)
        
        t_m = tf.reduce_mean(self.t_act_q, axis=1, keep_dims=True)
        t_devs_squared = tf.square(self.t_act_q - t_m)
        t_std = tf.reduce_mean(t_devs_squared, axis=1, keep_dims=True)
        
        
        ### 理論上這邊第二項要standarize (減掉平均值 再除以 標準差),但反而會train壞,有待改進
        self.e_out = self.e_value + self.e_act_q
        self.t_out = self.t_value + self.t_act_q
        
        
        ######### double QDN只要改這邊 #########
        # 用eval net得到的batch action,再利用target net得到batch action所對應的Q值
        # 會加速不少
        
        e_actions = tf.argmax(self.e_out,axis=1,output_type=tf.int32)
        e_actions_idx = tf.stack([tf.range(tf.shape(e_actions)[0]),e_actions],axis=1)
        next_q=tf.gather_nd(params=self.t_out,indices = e_actions_idx)
        
        ######################################
        
        self.target_q = self.r + self.gamma*next_q
#         self.target_q = tf.stop_gradient(self.target_q)  # target不要更新參數
        
        index = tf.stack([tf.range(tf.shape(self.a)[0]),self.a],axis=1)
        self.eval_q = tf.gather_nd(params = self.e_out , indices = index)
        self.loss = tf.reduce_mean(tf.squared_difference(self.eval_q,self.target_q))
        
        # 只更新eval net的參數
        q_vars = [ var for var in tf.trainable_variables() if "eval_net" in var.name]
        self.step = tf.train.RMSPropOptimizer(learning_rate = self.lr).minimize(self.loss,var_list=q_vars)
        
    
        
    def choose_action(self,states):
        states = states[np.newaxis,:]
        
        q_val = self.sess.run(self.e_out,feed_dict={self.s:states})
        if np.random.normal() < self.episilon:
            act = np.argmax(q_val,axis=1)[0]
        else:
            act = np.random.choice(self.n_actions)
        return act
        
    def store_entry(self,s,a,r,s_):
        idx = self.memory_counter % self.memory_capacity
        
        row_entry = np.hstack([s,a,r,s_])
        self.memory[idx,:] = row_entry
        
        self.memory_counter+=1
        
        
    def train(self):
        idx = np.random.choice(self.memory_capacity, self.batch_size)
        memory_batch = self.memory[idx,:]
        
        s = memory_batch[:,:self.n_states]
        a = memory_batch[:,self.n_states]
        r = memory_batch[:,self.n_states+1]
        s_ = memory_batch[:,-self.n_states:]
        
        loss,_ = self.sess.run([self.loss,self.step],feed_dict={self.s:s,
                                                               self.a:a,
                                                               self.r:r,
                                                               self.s_:s_})

        
        
        # 固定步數執行同步動作
        if self.synchronize_counter%self.synchronize_iter==0:
            print("eval參數 同步到 target參數")
            self.sess.run(self.synchronize_op)
            
        self.synchronize_counter+=1

In [56]:
DQN = dueling_double_DQN_Agent()

In [57]:
env = gym.make('CartPole-v0')
env = env.unwrapped

In [58]:
for i_episode in range(70):
    s = env.reset()
    ep_r = 0
    while True:
        env.render()
        a = DQN.choose_action(s)
#         print(a)
        # take action
        s_, r, done, info = env.step(a)

        # modify the reward
        x, x_dot, theta, theta_dot = s_
        
    
        # reward要重新定義 預設是 : Reward is 1 for every step taken, including the termination step
        ## env.x_threshold代表x方向的最大距離,r1越大代表越靠近中間
        ## -0.8是要讓reward 不要那麼通膨, 最中間也只能拿 1-0.8 = 0.2 reward
        ## -0.8不加也是可以train的起來的
        r1 = (env.x_threshold - abs(x)) / env.x_threshold -0.8
        
        ## 角度越接近正垂直，r2越大
        ## -0.5同上是修正值
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians -0.5
        r = r1 + r2

        DQN.store_entry(s, a, r, s_)

        ep_r += r
        if DQN.memory_counter > DQN.memory_capacity:
            DQN.train()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))

        if done:
            break
        s = s_
env.close()

eval參數 同步到 target參數
Ep:  3 | Ep_r:  -3.29
Ep:  4 | Ep_r:  1.86
Ep:  5 | Ep_r:  2.06
Ep:  6 | Ep_r:  1.08
Ep:  7 | Ep_r:  1.03
Ep:  8 | Ep_r:  4.13
Ep:  9 | Ep_r:  1.07
Ep:  10 | Ep_r:  1.56
Ep:  11 | Ep_r:  1.04
Ep:  12 | Ep_r:  2.99
eval參數 同步到 target參數
Ep:  13 | Ep_r:  2.55
Ep:  14 | Ep_r:  1.64
Ep:  15 | Ep_r:  9.08
Ep:  16 | Ep_r:  2.74
Ep:  17 | Ep_r:  2.89
Ep:  18 | Ep_r:  1.96
eval參數 同步到 target參數
Ep:  19 | Ep_r:  5.63
Ep:  20 | Ep_r:  4.18
Ep:  21 | Ep_r:  1.2
Ep:  22 | Ep_r:  -1.19
eval參數 同步到 target參數
Ep:  23 | Ep_r:  -3.76
Ep:  24 | Ep_r:  -2.37
eval參數 同步到 target參數
Ep:  25 | Ep_r:  2.36
Ep:  26 | Ep_r:  12.79
eval參數 同步到 target參數
Ep:  27 | Ep_r:  8.71
Ep:  28 | Ep_r:  3.97
Ep:  29 | Ep_r:  4.58
Ep:  30 | Ep_r:  2.65
Ep:  31 | Ep_r:  3.94
Ep:  32 | Ep_r:  2.87
Ep:  33 | Ep_r:  2.17
Ep:  34 | Ep_r:  2.77
Ep:  35 | Ep_r:  1.4
eval參數 同步到 target參數
Ep:  36 | Ep_r:  3.35
Ep:  37 | Ep_r:  0.59
eval參數 同步到 target參數
Ep:  38 | Ep_r:  23.21
eval參數 同步到 target參數
Ep:  39 | Ep_r:  60.32
eval參數 同